In [1]:
!pip install pyspark


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import pyspark

In [127]:
from pyspark.sql import SparkSession

In [128]:
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [5]:
df = spark.read.csv("data/raw/Dataset/actions.csv")
df.show(5)

+----+-------+---------+------------------+-------+---------+-------+------------------+-----+------------------+-----------------+-----------+-------+---------+---------+---------+-----------+-------------+
| _c0|    _c1|      _c2|               _c3|    _c4|      _c5|    _c6|               _c7|  _c8|               _c9|             _c10|       _c11|   _c12|     _c13|     _c14|     _c15|       _c16|         _c17|
+----+-------+---------+------------------+-------+---------+-------+------------------+-----+------------------+-----------------+-----------+-------+---------+---------+---------+-----------+-------------+
|NULL|game_id|period_id|      time_seconds|team_id|player_id|start_x|           start_y|end_x|             end_y|original_event_id|bodypart_id|type_id|result_id|action_id|type_name|result_name|bodypart_name|
|   0|2500089|        1|2.7635970000000043|   1659|     9637|   52.5|              34.0| 63.0|              30.6|        251700146|          0|      0|        1|       

In [7]:
events_df = spark.read.csv("data/raw/Dataset/events_England.csv")
events_df.show()

+-------+------------+--------------------+--------+--------------------+-------+---------+------+-----------+------------------+----------+---------+------------+----------+----------+----------+----------+
|    _c0|         _c1|                 _c2|     _c3|                 _c4|    _c5|      _c6|   _c7|        _c8|               _c9|      _c10|     _c11|        _c12|      _c13|      _c14|      _c15|      _c16|
+-------+------------+--------------------+--------+--------------------+-------+---------+------+-----------+------------------+----------+---------+------------+----------+----------+----------+----------+
|eventId|subEventName|                tags|playerId|           positions|matchId|eventName|teamId|matchPeriod|          eventSec|subEventId|       id|    tagsList|pos_orig_y|pos_orig_x|pos_dest_y|pos_dest_x|
|      8| Simple pass|      [{'id': 1801}]|   25413|[{'y': 49, 'x': 4...|2499719|     Pass|  1609|         1H|2.7586489999999912|      85.0|177959171|      [1801]|     

In [131]:
# We will read the csv file into a dataframe
# But the headers has "." on some of the name which will throw error later on
# to minimize this risk we will replace it with underscore.

def replace_character(col_name: str) -> str:
    col_name = col_name.replace(".", "_")
    return col_name

matches = spark.read.csv("data/raw/Dataset/matches_England.csv", header=True) 
new_columns = [replace_character(column) for column in matches.columns]
matches = matches.toDF(*new_columns)
matches.show()


# Row(_c0='status', _c1='roundId', _c2='gameweek', _c3='teamsData', _c4='seasonId', 
# _c5='dateutc', _c6='winner', _c7='venue', _c8='wyId', _c9='label', _c10='date', _c11='referees',
# _c12='duration', _c13='competitionId', _c14='team1.scoreET', _c15='team1.coachId', _c16='team1.side', 
# _c17='team1.teamId', _c18='team1.score', _c19='team1.scoreP', _c20='team1.hasFormation', _c21='team1.formation', 
# _c22='team1.scoreHT', _c23='team1.formation.bench', _c24='team1.formation.lineup', _c25='team1.formation.substitutions', 
# _c26='team2.scoreET', _c27='team2.coachId', _c28='team2.side', _c29='team2.teamId', _c30='team2.score', _c31='team2.scoreP',
#  _c32='team2.hasFormation', _c33='team2.formation', _c34='team2.scoreHT', _c35='team2.formation.bench', 
# _c36='team2.formation.lineup', _c37='team2.formation.substitutions')
# matches.select(['_c9', '_c10', lit(matches.team1.score + matches.team2.score).alias("totalGoals")]).show()

+------+-------+--------+--------------------+--------+-------------------+------+--------------------+-------+--------------------+--------------------+--------------------+--------+-------------+-------------+-------------+----------+------------+-----------+------------+------------------+--------------------+-------------+---------------------+----------------------+-----------------------------+-------------+-------------+----------+------------+-----------+------------+------------------+--------------------+-------------+---------------------+----------------------+-----------------------------+
|status|roundId|gameweek|           teamsData|seasonId|            dateutc|winner|               venue|   wyId|               label|                date|            referees|duration|competitionId|team1_scoreET|team1_coachId|team1_side|team1_teamId|team1_score|team1_scoreP|team1_hasFormation|     team1_formation|team1_scoreHT|team1_formation_bench|team1_formation_lineup|team1_formation_s

In [125]:
# lets create a custom query dataframe

from pyspark.sql.functions import col

# select required columns
most_goals_df = matches.where(matches.status == 'Played') \
                .select("date", "label", (matches.team1_score + matches.team2_score).alias("totalGoals")) \
                    
most_goals_df.sort([most_goals_df.totalGoals], ascending=False).show(truncate=False)

+--------------------------------------+--------------------------------------------+----------+
|date                                  |label                                       |totalGoals|
+--------------------------------------+--------------------------------------------+----------+
|May 13, 2018 at 4:00:00 PM GMT+2      |Tottenham Hotspur - Leicester City, 5 - 4   |9.0       |
|October 14, 2017 at 4:00:00 PM GMT+2  |Manchester City - Stoke City, 7 - 2         |9.0       |
|August 11, 2017 at 8:45:00 PM GMT+2   |Arsenal - Leicester City, 4 - 3             |7.0       |
|January 14, 2018 at 5:00:00 PM GMT+1  |Liverpool - Manchester City, 4 - 3          |7.0       |
|December 26, 2017 at 1:30:00 PM GMT+1 |Tottenham Hotspur - Southampton, 5 - 2      |7.0       |
|October 22, 2017 at 2:30:00 PM GMT+2  |Everton - Arsenal, 2 - 5                    |7.0       |
|August 12, 2017 at 1:30:00 PM GMT+2   |Watford - Liverpool, 3 - 3                  |6.0       |
|December 22, 2017 at 8:45:00 

In [132]:
spark.stop()